# Text Analysis on sydneysiege data

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [3]:
df.columns

Index(['src_tweet_id', 'src_user_id', 'src_tweet', 'src_date',
       'reply_tweet_id', 'reply_user_id', 'reply_tweet', 'reply_date',
       'label'],
      dtype='object')

In [4]:
df

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544270066661851136,27735195,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:17:50 +0000 2014,0
1,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544273361334194176,14875436,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:30:56 +0000 2014,0
2,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544275366588588032,14121167,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:38:54 +0000 2014,0
3,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544276615316770816,187877777,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:43:51 +0000 2014,0
4,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544278115589976065,372216479,RT “@nswpolice: A police operation is underway...,Sun Dec 14 23:49:49 +0000 2014,0
...,...,...,...,...,...,...,...,...,...
22770,544521166543548416,87261221,"Sadly, at this stage, there are two confirmed ...",Mon Dec 15 15:55:37 +0000 2014,544522905082540032,2159158525,"“@PoliceAU: Sadly, at this stage, there are tw...",Mon Dec 15 16:02:31 +0000 2014,1
22771,544521166543548416,87261221,"Sadly, at this stage, there are two confirmed ...",Mon Dec 15 15:55:37 +0000 2014,544522916042252289,2159158525,"“@PoliceAU: Sadly, at this stage, there are tw...",Mon Dec 15 16:02:34 +0000 2014,1
22772,544521166543548416,87261221,"Sadly, at this stage, there are two confirmed ...",Mon Dec 15 15:55:37 +0000 2014,544522948850094081,450436433,@PoliceAU no :'(,Mon Dec 15 16:02:42 +0000 2014,1
22773,544521166543548416,87261221,"Sadly, at this stage, there are two confirmed ...",Mon Dec 15 15:55:37 +0000 2014,544522997466267648,450436433,@PoliceAU nonono,Mon Dec 15 16:02:54 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [5]:
import pickle

In [6]:
'''Convert Emojis to Words'''

with open(r'D:\論文\PHEME9\Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    import re
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

In [7]:
def textprocessing(text):
    
    text = convert_emojis_to_word(text)   #convert emoji to word
    
    text = text.lower()            #lower case
     
    quo = text.find('“')           #cut quoting part
    quo_end = text.find('”')
    if quo != -1:
        if quo_end != -1:
            text = text[0: quo:] + text[quo_end+1: :]
        else:
            quo_end2 = text.find('“',quo+1)
            text = text[0: quo:] + text[quo_end2+1: :]

    text = " ".join([word for word in text.split() if 'http' not in word    #remove url
                                and not word.startswith('@')                
                                and word != 'rt'])   
                 
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))

    from nltk.tokenize import word_tokenize
    text = word_tokenize(text)
   
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    stop_words=set([a for a in list(stop_words) if re.search("n't|no|not",a) ==None])
    stop_words.add('now')
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    
    from nltk.stem.porter import PorterStemmer
    stem = PorterStemmer()
    text = [stem.stem(i) for i in text]
    
    text = " ".join(text)
    
    return text

In [8]:
print("original text: ", df.reply_tweet.iloc[0])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[0]))

original text:  “@nswpolice: A police operation is underway in Martin Place, Sydney's CBD. People are advised to avoid the area.”
cleaned text:  


In [10]:
df[df.reply_tweet.str.find("“") != -1].reply_tweet
# df[df.reply_tweet.str.find('“'or'”') != -1].reply_tweet

0        “@nswpolice: A police operation is underway in...
2        “@nswpolice: A police operation is underway in...
4        RT “@nswpolice: A police operation is underway...
9        Scary stuff “@abcnewsSydney #BREAKING A police...
17       “@SkyNews: Australian Police Operation Under W...
                               ...                        
22765    “@PoliceAU: Sadly, at this stage, there are tw...
22766    “@PoliceAU: Sadly, at this stage, there are tw...
22768    “@PoliceAU: Sadly, at this stage, there are tw...
22770    “@PoliceAU: Sadly, at this stage, there are tw...
22771    “@PoliceAU: Sadly, at this stage, there are tw...
Name: reply_tweet, Length: 1431, dtype: object

### applied on original data (**df**)

In [11]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [12]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544270066661851136,27735195,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:17:50 +0000 2014,0,polic oper underway martin place sydney cbd pe...,
1,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544273361334194176,14875436,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:30:56 +0000 2014,0,polic oper underway martin place sydney cbd pe...,polic oper underway martin place sydney cbd pe...
2,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544275366588588032,14121167,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:38:54 +0000 2014,0,polic oper underway martin place sydney cbd pe...,sicken
3,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544276615316770816,187877777,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:43:51 +0000 2014,0,polic oper underway martin place sydney cbd pe...,polic oper underway martin place sydney cbd pe...
4,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544278115589976065,372216479,RT “@nswpolice: A police operation is underway...,Sun Dec 14 23:49:49 +0000 2014,0,polic oper underway martin place sydney cbd pe...,


## Sentiment Analysis

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [14]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [15]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [16]:
df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw,src_sentiment,reply_sentiment
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544270066661851136,27735195,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:17:50 +0000 2014,0,polic oper underway martin place sydney cbd pe...,,Negative,Neutral


### Source tweet data

In [17]:
src_tw_df = df.iloc[:,[0,1,2,3,8,9,11]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df.reset_index(drop=True,inplace=True)
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label,cleaned_src_tw,src_sentiment
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,0,polic oper underway martin place sydney cbd pe...,Negative
1,544268900351766528,16215478,#BREAKING A police operation is underway in th...,Sun Dec 14 23:13:12 +0000 2014,0,break polic oper underway heart sydney cbd one...,Negative
2,544274797640044544,7587032,Australian Police Operation Under Way In Sydne...,Sun Dec 14 23:36:38 +0000 2014,0,australian polic oper way sydney,Neutral
3,544275574642835456,27964284,Prayers needed for the situation happening in ...,Sun Dec 14 23:39:43 +0000 2014,0,prayer need situat happen sydney right,Neutral
4,544275733607366656,374712154,This is not going to end well in Sydney . Wake...,Sun Dec 14 23:40:21 +0000 2014,0,not go end well sydney wake britain come,Negative
...,...,...,...,...,...,...,...
1168,544520042810200064,442653867,"#sydneysiege is over. 2 confirmed dead, #PrayF...",Mon Dec 15 15:51:09 +0000 2014,1,sydneysieg 2 confirm dead prayforsydney prayfo...,Negative
1169,544520273718812672,100629223,Police confirm that #sydneysiege is finally ov...,Mon Dec 15 15:52:04 +0000 2014,1,polic confirm sydneysieg final two peopl repor...,Negative
1170,544520932405284864,7587032,Seven people reportedly taken away on stretche...,Mon Dec 15 15:54:41 +0000 2014,1,seven peopl reportedli taken away stretcher pa...,Positive
1171,544521141335764992,342894187,BREAKING: CNN: Sydney police storm cafe. 2 con...,Mon Dec 15 15:55:31 +0000 2014,1,break cnn sydney polic storm cafe 2 confirm de...,Negative


### save data to csv 

In [ ]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df.csv',index=False)
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/sydneysiege/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/sydneysiege/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/sydneysiege/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter
freq = FreqDist(processed_text)
freq.plot(20)

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [18]:
import text2emotion as te

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [20]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['reply_emo_te'] = emo

In [21]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df.csv',index=False)

In [22]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['src_emo_te']=emo1

In [23]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src.csv',index=False)